# ENV

In [ ]:
%pip install 'git+https://github.com/facebookresearch/detectron2.git'
#!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu110/torch1.7/index.html
#!python -m pip install detectron2==0.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu110/torch1.7/index.html

In [ ]:
#!pip install albumentations==0.5.2 pyyaml==5.1

In [ ]:
!mkdir models logs configs 

In [ ]:
!nvidia-smi

# Imports and utils

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from typing import List, Optional, Union
from detectron2.data.transforms import Transform as T
import torch
import torchvision
import albumentations as A
#from albumentations.augmentations.transforms import Sharpen
#from albumentations.pytorch.transforms import ToTensorV2


import detectron2
from detectron2.data.datasets import register_coco_instances
from detectron2.data import transforms as T
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog, DatasetMapper, build_detection_test_loader , build_detection_train_loader
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import detection_utils as utils
from detectron2.config import configurable
from detectron2.engine.hooks import EvalHook
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.utils.logger import setup_logger
setup_logger()

from sklearn.model_selection import KFold,StratifiedKFold,StratifiedShuffleSplit,GroupKFold

import matplotlib.pyplot as plt
import cv2
import json
import io
import os
import copy
import random
from IPython.display import FileLink, FileLinks
import yaml
from abc import ABC,ABCMeta, abstractmethod

# Config

In [ ]:
class Config(metaclass=ABCMeta):
    general={
        "DATASET_PATH" : "../input/tacotrashdataset",
        "LOGS_PATH" : "logs",
        "MODELS_PATH" : "models",
        "CONFIG_PATH" : "configs",
        "seed" : 42,
        "n_folds" : 5,
        "tool" : "detectron2",
        "experiment_id" : "24-03-2021",
        "category" : "super_category",
        "augmentations" : True,
        "TTA" : False
        
    }
    model={
        "base_lr" : 0.001,
        "num_classes" : 29, #29 if super category 60 if normal category 
        "model_name" : "faster_rcnn_R_101_FPN_3x",
        "batchsize_per_image" : 1024,
        "images_per_batch" : 4,
        "epochs" : 9,
    }

    @staticmethod
    def dump(config,path="config.yaml"):
        stream = open(path, 'w')
        yaml.dump(config,stream)
        stream.close()
    
    @staticmethod
    def load(path="config.yaml"):
        stream = open(path, 'r')
        config=yaml.load(stream)
        stream.close()
        return config

global_config=Config()
c=global_config
def inject_config(funct):
    def function_wrapper(*args,**kwargs):
        return funct(global_config,*args,**kwargs)  
    return function_wrapper
    

In [ ]:
Config.dump(global_config.general,os.path.join(global_config.general["CONFIG_PATH"],"global_config.yaml"))
Config.dump(global_config.model,os.path.join(global_config.general["CONFIG_PATH"],"model_config.yaml"))

# SEED

In [ ]:
@inject_config
def seed_all(config):
    seed_value=config.general["seed"]
    random.seed(seed_value) # Python
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False

# FIX annotation duplicated ids and negative bboxes

repeated annotations idx 

308 => 0

4039  =>2197


In [ ]:
annot=json.load(open(os.path.join(global_config.general["DATASET_PATH"],"data/annotations.json")))
annot["annotations"][308]["id"]=0
annot["annotations"][4039]["id"]=2197

## delete negative BBOX

In [ ]:
annot_to_delete=[]
for idx,annotation in enumerate(annot["annotations"]):
    if (annotation["bbox"][0]<0 or annotation["bbox"][1]<0 or
        annotation["bbox"][2]<0 or annotation["bbox"][3]<0):
        annot_to_delete.append(idx)
for pos,idx in enumerate(annot_to_delete):
    del annot["annotations"][idx-pos]


In [ ]:
json.dump(annot,open("new_annotations.json","w"))

# Annotation Preprocess and Viz

## Categories and Super categories dict

In [ ]:
categories={ annotation["id"] : annotation["name"] for annotation in annot["categories"]}
super_categories={ annotation["id"] : annotation["supercategory"] for annotation in annot["categories"]}

## Preprocess

In [ ]:
annot_df=pd.DataFrame(annot["annotations"])
images_df=pd.DataFrame(annot["images"])

In [ ]:
images_df.describe()

In [ ]:
"""
for code testing
"""
#annot_df=annot_df.head(200)

In [ ]:
annot_df["category"]=annot_df["category_id"].apply(lambda value : categories[value])
annot_df["super_category"]=annot_df["category_id"].apply(lambda value : super_categories[value])
super_category_to_index={value : key for key,value in enumerate(annot_df["super_category"].unique())}
annot_df["super_category_id"]=annot_df["super_category"].apply(lambda value : super_category_to_index[value])
annot_df["normal_category_id"]=annot_df["category_id"]
annot_df["normal_category"]=annot_df["category"]
if c.general["category"] != "normal_category":
    annot_df["category_id"]=annot_df["super_category_id"]
    annot_df["category"]=annot_df["super_category"]
    annot_cat=annot_df.groupby("category_id")[["category_id","category","super_category"]].first()
    annot_cat.columns=["id","name","supercategory"]
    annot["categories"]=annot_cat.to_dict("records")

                       

## VIZ

In [ ]:
annot_df.head()

In [ ]:
annot_df["category"].value_counts().head(10).plot(kind="bar",title="annotations category distribution")

In [ ]:
annot_df["super_category"].value_counts().head(10).plot(kind="bar",title="annotations super category distribution")

# Kfold

In [ ]:
@inject_config
def kfold_split(config):
    seed_all()
    annot_df["folds"]=-1
    kf = GroupKFold(n_splits=config.general["n_folds"])
    #kf = StratifiedShuffleSplit(n_splits=Config.n_folds,test_size=Config.test_size, random_state=Config.random_state, )
    #kf = StratifiedShuffleSplit(n_splits=10,test_size=0.1, random_state=Config.random_state, )

    for fold, (_, val_index) in enumerate(kf.split(annot_df,groups=annot_df["image_id"])):
            annot_df.loc[val_index, "folds"] = fold

In [ ]:
kfold_split()

In [ ]:
annot_df[annot_df["folds"]==0]["category"].value_counts().head(10).plot(kind="bar",title="Test annotations category distribution")

In [ ]:
annot_df[annot_df["folds"]!=0]["category"].value_counts().head(10).plot(kind="bar",title="Train annotations category distribution")

In [ ]:
annot_df[annot_df["folds"]==0]["super_category"].value_counts().head(10).plot(kind="bar",title="Test annotations super category distribution")

In [ ]:
annot_df[annot_df["folds"]==0]["super_category"].value_counts().head(10).plot(kind="bar",title="Train annotations super category distribution")

# Register Dataset

In [ ]:
@inject_config
def register_dataset(config,fold):
    train_dataset_name=f"my_dataset_train_{fold}"
    test_dataset_name=f"my_dataset_test_{fold}"
    train_dataset_file=f"my_dataset_train_{fold}.json"
    test_dataset_file=f"my_dataset_test_{fold}.json"
    
    train_annot_df=annot_df[annot_df["folds"]!=fold]
    test_annot_df=annot_df[annot_df["folds"]==fold]
    train_annot_df=train_annot_df.drop(["normal_category","normal_category_id"],axis=1)
    test_annot_df=test_annot_df.drop(["normal_category","normal_category_id"],axis=1)

    train_images_df=images_df[images_df["id"].apply(lambda i:True if i in list(train_annot_df["image_id"].unique()) else False)]
    test_images_df=images_df[images_df["id"].apply(lambda i:True if i in list(test_annot_df["image_id"].unique()) else False)]
    
    train_annot=annot.copy()
    test_annot=annot.copy()
    
    train_annot["annotations"]=train_annot_df.reset_index(drop=True).to_dict("records")
    train_annot["images"]=train_images_df.reset_index(drop=True).to_dict("records")
    test_annot["annotations"]=test_annot_df.reset_index(drop=True).to_dict("records")
    test_annot["images"]=test_images_df.reset_index(drop=True).to_dict("records")
    
    json.dump(train_annot,open(train_dataset_file,"w"))
    json.dump(test_annot,open(test_dataset_file,"w"))
    
    if train_dataset_name in DatasetCatalog.list():
        DatasetCatalog.remove(train_dataset_name)
        MetadataCatalog.remove(train_dataset_name)
    if test_dataset_name in DatasetCatalog.list():
        DatasetCatalog.remove(test_dataset_name)
        MetadataCatalog.remove(test_dataset_name)
        
    register_coco_instances(train_dataset_name, {}, train_dataset_file, os.path.join(config.general["DATASET_PATH"],"data"))
    register_coco_instances(test_dataset_name, {}, test_dataset_file, os.path.join(config.general["DATASET_PATH"],"data"))


# Preprocess and augmentations 

In [ ]:
def mono_to_color(X, mean=None, std=None, norm_max=None, norm_min=None, eps=1e-6,**kwargs):
    # Stack X as [X,X,X]
    #print(X.shape)
    #X = np.squeeze(np.stack([X, X, X], axis=-2),axis=-1)
    X=X.transpose(1, 0, 2)
    mean = mean or X.mean()
    std = std or X.std()
    Xstd = (X - mean) / (std + eps)
    _min, _max = Xstd.min(), Xstd.max()
    norm_max = norm_max or _max
    norm_min = norm_min or _min
    if (_max - _min) > eps:
        # Normalize to [0, 255]
        V = Xstd
        V[V < norm_min] = norm_min
        V[V > norm_max] = norm_max
        V = 255 * (V - norm_min) / (norm_max - norm_min)
        V = V.astype(np.uint8)
    else:
        # Just zero
        V = np.zeros_like(Xstd, dtype=np.uint8)
    return V
  

class ToColor:
    def __init__(self,
                  mean=None,
                  std=None):
        self.mean=mean
        self.std = std
        

    def __call__(self, image,**kwargs):
        return mono_to_color(image,
                            self.mean,
                            self.std,
                            )

In [ ]:
def get_train_transforms():
    return A.Compose(
        [
            #A.RandomSizedCrop(min_max_height=(1024, 1024), height=1024, width=1024, p=0.2),
            A.OneOf([
                A.HueSaturationValue(hue_shift_limit=0.4, sat_shift_limit= 0.4, 
                                     val_shift_limit=0.4, p=0.8),
                A.RandomBrightnessContrast(brightness_limit=0.4, 
                                           contrast_limit=0.4, p=0.8),
            ],p=0.8),
            A.Rotate (limit=15, interpolation=1, border_mode=4, value=None, mask_value=None, p=0.8),
            
            #A.transforms.Sharpen (alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=0.5),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.RandomResizedCrop (1500, 1500, scale=(0.8, 0.8), ratio=(0.75, 1.3333333333333333), interpolation=1, always_apply=False, p=0.3),
            A.Resize(height=1500, width=1500, p=1.0),
            A.CLAHE(p=0.7),
            #A.Lambda(ToColor(),p=1),
            #A.Cutout(num_holes=16, max_h_size=64, max_w_size=64, fill_value=0, p=0.1),
            
        ], 
        p=1.0, 
        bbox_params=A.BboxParams(
            format='coco',
            min_area=0.5, 
            min_visibility=0.5,
            label_fields=['category_id']
        )
    )

def get_valid_transforms():
    return A.Compose(
        [
            
            A.Resize(height=1500, width=1500, p=1.0),
            A.CLAHE(p=1),
            #A.Normalize(p=1),
            #A.Lambda(ToColor(),p=1),
            
        ], 
        p=1.0, 
        bbox_params=A.BboxParams(
            format='coco',
            min_area=0.5, 
            min_visibility=0.5,
            label_fields=['category_id']
        )
    )

def get_transforms(train=True):
    if (train):
        return get_train_transforms()
    return get_valid_transforms()
albu_transformations=get_transforms(train=True)
#albu_transformations=None

In [ ]:
class PersonalMapper (detectron2.data.DatasetMapper):
    def __call__(self, dataset_dict):
        """
        Args:
            dataset_dict (dict): Metadata of one image, in Detectron2 Dataset format.

        Returns:
            dict: a format that builtin models in detectron2 accept
        """
        dataset_dict = copy.deepcopy(dataset_dict)  # it will be modified by code below
        # USER: Write your own image loading if it's not from a file
        image = utils.read_image(dataset_dict["file_name"], format=self.image_format)
        #utils.check_image_size(dataset_dict, image)

        
        
        ##### ADDED PART

        #print("dataset dict : ",dataset_dict)

        annos = [
            obj for obj in dataset_dict["annotations"]
        ]
        annos_bbox = [
            obj["bbox"] for obj in dataset_dict["annotations"]
        ]
        annos_categroy_id = [
            obj["category_id"] for obj in dataset_dict.pop("annotations")
        ]
        
        if albu_transformations is not None:
            transform_list=get_transforms(self.is_train)
            image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            transform_result=transform_list(image=image,bboxes=annos_bbox,category_id=annos_categroy_id)
            image=cv2.cvtColor(transform_result["image"], cv2.COLOR_RGB2BGR)
            annos=[annos[i] for i in range(len(transform_result["bboxes"]))]
            for i in range(len(annos)):
                annos[i]["bbox"]=list(transform_result["bboxes"][i])
                annos[i]["category_id"]=transform_result["category_id"][i]
        
        dataset_dict["annotations"]=annos
        
        
        ##### ADDED PART
        
        # USER: Remove if you don't do semantic/panoptic segmentation.
        if "sem_seg_file_name" in dataset_dict:
            sem_seg_gt = utils.read_image(dataset_dict.pop("sem_seg_file_name"), "L").squeeze(2)
        else:
            sem_seg_gt = None

        aug_input = T.AugInput(image, sem_seg=sem_seg_gt)
        transforms = self.augmentations(aug_input)
        image, sem_seg_gt = aug_input.image, aug_input.sem_seg

        image_shape = image.shape[:2]  # h, w
        # Pytorch's dataloader is efficient on torch.Tensor due to shared-memory,
        # but not efficient on large generic data structures due to the use of pickle & mp.Queue.
        # Therefore it's important to use torch.Tensor.
        dataset_dict["image"] = torch.as_tensor(np.ascontiguousarray(image.transpose(2, 0, 1)))
        if sem_seg_gt is not None:
            dataset_dict["sem_seg"] = torch.as_tensor(sem_seg_gt.astype("long"))

        # USER: Remove if you don't use pre-computed proposals.
        # Most users would not need this feature.
        if self.proposal_topk is not None:
            utils.transform_proposals(
                dataset_dict, image_shape, transforms, proposal_topk=self.proposal_topk
            )

        if not self.is_train:
            # USER: Modify this if you want to keep them for some reason.
            dataset_dict.pop("annotations", None)
            dataset_dict.pop("sem_seg_file_name", None)
            return dataset_dict

        if "annotations" in dataset_dict:
            # USER: Modify this if you want to keep them for some reason.
            for anno in dataset_dict["annotations"]:
                if not self.use_instance_mask:
                    anno.pop("segmentation", None)
                if not self.use_keypoint:
                    anno.pop("keypoints", None)

            # USER: Implement additional transformations if you have other types of data
            annos = [
                utils.transform_instance_annotations(
                    obj, transforms, image_shape, keypoint_hflip_indices=self.keypoint_hflip_indices
                )
                for obj in dataset_dict.pop("annotations")
                if obj.get("iscrowd", 0) == 0
            ]
            instances = utils.annotations_to_instances(
                annos, image_shape, mask_format=self.instance_mask_format
            )

            # After transforms such as cropping are applied, the bounding box may no longer
            # tightly bound the object. As an example, imagine a triangle object
            # [(0,0), (2,0), (0,2)] cropped by a box [(1,0),(2,2)] (XYXY format). The tight
            # bounding box of the cropped triangle should be [(1,0),(2,1)], which is not equal to
            # the intersection of original bounding box and the cropping box.
            if self.recompute_boxes:
                instances.gt_boxes = instances.gt_masks.get_bounding_boxes()
            dataset_dict["instances"] = utils.filter_empty_instances(instances)
        return dataset_dict

In [ ]:
def show_rand_image(train_ds,meta_ds):
    """
    Can't find solution to show image with augs
    """
    mapper=PersonalMapper(cfg,is_train=True, augmentations=[])
    plt.figure(figsize=(16,20))
    for d in random.sample(ds, 1):
        d=mapper(d)
        d["annotations"]=[]
        #print("d : ",d["instances"].get_fields())

        for bbox,classes in zip(d["instances"].get_fields()["gt_boxes"],
                        d["instances"].get_fields()["gt_classes"]):
            #print("bbox : ",bbox.numpy())
            instance={
                'iscrowd': 0,
                'bbox': list(bbox.numpy()),
                'bbox_mode': detectron2.structures.BoxMode.XYXY_ABS,
                'category_id' : classes
            }
            d["annotations"].append(instance)
        #print("annotations : ",d["annotations"])
        img = d["image"]
        visualizer = Visualizer(img.numpy().transpose(1, 2, 0), metadata=meta, scale=0.5)
        out = visualizer.draw_dataset_dict(d)
        plt.imshow(out.get_image()[:, :, ::-1])

In [ ]:
#show_rand_image(ds,meta)

# STATS and VIZ

In [ ]:
meta_df=pd.read_csv("../input/tacotrashdataset/meta_df.csv")

In [ ]:
register_coco_instances("my_dataset", {}, "new_annotations.json", os.path.join(global_config.general["DATASET_PATH"],"data"))

In [ ]:
ds=DatasetCatalog.get("my_dataset")
meta=MetadataCatalog.get("my_dataset")

In [ ]:
plt.figure(figsize=(16,20))
for d in random.sample(ds, 1):
    #print("d : ",d)
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=meta, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    plt.imshow(out.get_image()[:, :, ::-1])

In [ ]:
#show_rand_image(ds,meta)

# Build Trainer

In [ ]:
class PersonalTrainer (detectron2.engine.defaults.DefaultTrainer):
    
    def __init__(self, cfg , config=c):
        super().__init__(cfg)
        self.metric=0
        self.checkpointer.save_dir=config.general["MODELS_PATH"]

        
    def build_hooks(self):
        hooks = super().build_hooks()
        def save_best_model():
            
            metric=self.test(self.cfg, self.model)["bbox"]["AP50"]
            if(metric>self.metric):
                self.metric=metric
                self.checkpointer.save("best_model") # it will add .pth alone
                
        steps_per_epoch=annot_df.shape[0]//global_config.model["images_per_batch"]
        model_checkpointer=EvalHook(steps_per_epoch, save_best_model)
        hooks.insert(-1,model_checkpointer)
        return hooks
    
    @classmethod
    def build_train_loader(cls, cfg):
        
        #return build_detection_train_loader(cfg,mapper=DatasetMapper(cfg,is_train=True,))
        return build_detection_train_loader(cfg,mapper=PersonalMapper(cfg,is_train=True,augmentations=[]))
    @classmethod
    def build_test_loader(cls, cfg, dataset_name):
        
        #return build_detection_test_loader( cfg,dataset_name,mapper=DatasetMapper(cfg,is_train=False,))
        return build_detection_test_loader( cfg,dataset_name,mapper=PersonalMapper(cfg,is_train=False,augmentations=[]))

    
    @classmethod
    def build_evaluator(cls, cfg, dataset_name):
        return COCOEvaluator(dataset_name, ("bbox",), False, output_dir=None)

# Prepare config params

In [ ]:
@inject_config
def get_config(config,fold=0):
    steps_per_epoch=annot_df.shape[0]//config.model["images_per_batch"]
    train_dataset_name=f"my_dataset_train_{fold}"
    test_dataset_name=f"my_dataset_test_{fold}"
    cfg = get_cfg()
    cfg.MODEL.DEVICE='cuda' if torch.cuda.is_available() else 'cpu'
    cfg.merge_from_file(model_zoo.get_config_file(f"COCO-Detection/{config.model['model_name']}.yaml"))
    cfg.DATASETS.TRAIN = (train_dataset_name,)
    cfg.DATASETS.TEST = (test_dataset_name,)
    cfg.DATALOADER.NUM_WORKERS = 4
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(f"COCO-Detection/{config.model['model_name']}.yaml")  # Let training initialize from model zoo
    cfg.SOLVER.IMS_PER_BATCH = config.model["images_per_batch"]
    #cfg.SOLVER.IMS_PER_BATCH = 100
    cfg.SOLVER.BASE_LR = config.model["base_lr"]  # pick a good LR
    cfg.SOLVER.MAX_ITER = steps_per_epoch*config.model["epochs"]  # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
    #cfg.SOLVER.MAX_ITER = 100
    cfg.SOLVER.STEPS = (steps_per_epoch*8,)
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = config.model["batchsize_per_image"]   # faster, and good enough for this toy dataset (default: 512)
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = config.model["num_classes"]  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
    cfg.TEST.EVAL_PERIOD=0
    #cfg.TEST.EVAL_PERIOD=steps_per_epoch
    cfg.OUTPUT_DIR = config.general["LOGS_PATH"]
    #cfg.TEST.EVALUATION_PERIOD=95
    cfg.OUTPUT_DIR_BEST=f'{config.general["LOGS_PATH"]}'
    cfg.SOLVER.AMP.ENABLED = True
    cfg.MODEL.WEIGHTS = "../input/trash-detection-output/models/best_model.pth"

    cfg.SEED = config.general["seed"]

    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
    os.makedirs(cfg.OUTPUT_DIR_BEST, exist_ok=True)
    return cfg


In [ ]:
cfg=get_config()

# Prepare trainer

In [ ]:
@inject_config
def train(config,fold):
    seed_all()
    register_dataset(fold)
    cfg=get_config(fold)
    #trainer = DefaultTrainer(cfg)
    trainer = PersonalTrainer(cfg) 
    trainer.resume_or_load(resume=True)
    trainer.evaluator = COCOEvaluator(f"my_dataset_test_{fold}", ("bbox",), False, output_dir=None)
    trainer.train()
    

In [ ]:
train(0)

# Prepare Output

In [ ]:
with open(os.path.join(global_config.general["CONFIG_PATH"],"detectron_config.yaml"),"w") as f:
    f.write(get_config().dump())

In [ ]:
%ls logs

In [ ]:
%ls models

In [ ]:
%ls configs

In [ ]:
# Look at training curves in tensorboard:
#%load_ext tensorboard
#%tensorboard --logdir output

# Test Dataset

In [ ]:
metrics={}

In [ ]:
cfg=get_config()
#cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = os.path.join(global_config.general["MODELS_PATH"], "best_model.pth")  # path to the model we just trained
#cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "best_model.pth.pth")  # path to the model we just trained
#cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a custom testing threshold
model = build_model(cfg)
m=DetectionCheckpointer(model).load(cfg.MODEL.WEIGHTS)

In [ ]:
evaluator = COCOEvaluator("my_dataset_test_0", ("bbox",), False, output_dir=global_config.general["LOGS_PATH"])
val_loader = build_detection_test_loader(cfg, "my_dataset_test_0")
train_metric=inference_on_dataset(model, val_loader, evaluator)
metrics["train_metric"]=train_metric

In [ ]:
evaluator = COCOEvaluator("my_dataset_train_0", ("bbox",), False, output_dir=global_config.general["LOGS_PATH"])
val_loader = build_detection_test_loader(cfg, "my_dataset_train_0")
valid_metric=inference_on_dataset(model, val_loader, evaluator)
metrics["valid_metric"]=valid_metric

In [ ]:
Config.dump(metrics,os.path.join(global_config.general["LOGS_PATH"],"metrics.yaml"))